In [37]:
import os
import pandas as pd
from ast import literal_eval
import requests
from bs4 import BeautifulSoup

In [38]:
kaggle_wikilink = pd.read_csv('./dataset/kaggle_wikilink_all_finish.csv', \
                     index_col ='Column', converters={"Wiki_Link": literal_eval})

In [39]:
check_kaggle = kaggle_wikilink[kaggle_wikilink['tmp'] == 'check']

In [55]:
check_kaggle.head(5)

,Art_ID,Title,Artist,Wiki_Link,Style,Genre,Media,tmp,New_Wiki_Link,img_src
Column,,,,,,,,,,
7,Kaggle_49823,Small Bird on a Branch of Kaidozakura,Hiroshige,[https://www.wikiart.org/en/hiroshige/small-bi...,NaN,NaN,NaN,check,[https://www.wikiart.org/en/hiroshige/small-bi...,[https://uploads5.wikiart.org/images/hiroshige...
18,Kaggle_82352,The Garden of Earthly Delights,Hieronymus Bosch,[https://www.wikiart.org/en/hieronymus-bosch/t...,NaN,NaN,NaN,check,[https://www.wikiart.org/en/hieronymus-bosch/t...,[https://uploads2.wikiart.org/images/hieronymu...
20,Kaggle_85864,Untitled,Mark Rothko,[https://www.wikiart.org/en/mark-rothko/untitl...,NaN,NaN,NaN,check,[https://www.wikiart.org/en/mark-rothko/untitl...,[https://uploads3.wikiart.org/images/mark-roth...
37,Kaggle_15715,Self-portrait,Titian,[https://www.wikiart.org/en/titian/self-portra...,NaN,NaN,NaN,check,[https://www.wikiart.org/en/titian/self-portra...,[https://uploads5.wikiart.org/images/titian/se...
38,Kaggle_59486,Self Portrait,Gerrit Dou,[https://www.wikiart.org/en/gerrit-dou/self-po...,NaN,NaN,NaN,check,NaN,NaN


In [59]:
# check_kaggle_unique = check_kaggle.drop_duplicates(subset = ['Title', 'Artist'])

In [60]:
# check_kaggle_unique.head(10)

,Art_ID,Title,Artist,Wiki_Link,Style,Genre,Media,tmp,New_Wiki_Link,img_src
Column,,,,,,,,,,
7,Kaggle_49823,Small Bird on a Branch of Kaidozakura,Hiroshige,[https://www.wikiart.org/en/hiroshige/small-bi...,NaN,NaN,NaN,check,[https://www.wikiart.org/en/hiroshige/small-bi...,[https://uploads5.wikiart.org/images/hiroshige...
18,Kaggle_82352,The Garden of Earthly Delights,Hieronymus Bosch,[https://www.wikiart.org/en/hieronymus-bosch/t...,NaN,NaN,NaN,check,[https://www.wikiart.org/en/hieronymus-bosch/t...,[https://uploads2.wikiart.org/images/hieronymu...
20,Kaggle_85864,Untitled,Mark Rothko,[https://www.wikiart.org/en/mark-rothko/untitl...,NaN,NaN,NaN,check,[https://www.wikiart.org/en/mark-rothko/untitl...,[https://uploads3.wikiart.org/images/mark-roth...
37,Kaggle_15715,Self-portrait,Titian,[https://www.wikiart.org/en/titian/self-portra...,NaN,NaN,NaN,check,[https://www.wikiart.org/en/titian/self-portra...,[https://uploads5.wikiart.org/images/titian/se...
38,Kaggle_59486,Self Portrait,Gerrit Dou,[https://www.wikiart.org/en/gerrit-dou/self-po...,NaN,NaN,NaN,check,NaN,NaN
39,Kaggle_77921,Charity,Lucas Cranach the Elder,[https://www.wikiart.org/en/lucas-cranach-the-...,NaN,NaN,NaN,check,NaN,NaN
49,Kaggle_99733,Dipicting the characters from the Chushingura,Utagawa Kuniyoshi,[https://www.wikiart.org/en/utagawa-kuniyoshi/...,NaN,NaN,NaN,check,NaN,NaN
54,Kaggle_29331,Roger and Angelica,Odilon Redon,[https://www.wikiart.org/en/odilon-redon/roger...,NaN,NaN,NaN,check,NaN,NaN
55,Kaggle_53464,Divination,Konstantin Vasilyev,[https://www.wikiart.org/en/konstantin-vasilye...,NaN,NaN,NaN,check,NaN,NaN


In [64]:
check_kaggle_unique.to_csv('check_kaggle_unique.csv')

In [81]:
remove_sum = 0
# 한 행을 선택하고
for idx in check_kaggle_unique.index[3000:3500]:    
    try : 
        len(check_kaggle_unique.loc[idx, 'New_Wiki_Link'])
        continue
    except:
        pass
    
    print(idx)
    new_links = []
    imgs = []
    old_links = check_kaggle_unique.loc[idx, 'Wiki_Link']
    
    # 한 작품에 매칭되어있는 링크들 확인
    for link in old_links:
        response = requests.get(link)
        html = response.text
        
        soup = BeautifulSoup(html)
        
        try : 
            # media?
            media_check = 0
            article = soup.select_one('article')
            for info in article.select('.dictionary-values'):
                info = info.text
                key, value = info.split(':')

                if key.strip() == 'Media':
                    media_check = 1

            if media_check:
                new_links.append(link)

                img_src = soup.select_one('.wiki-layout-left-menu img')['src']
                imgs.append(img_src)
        except:
            pass
        
    check_kaggle_unique.loc[idx, 'New_Wiki_Link'] = new_links
    check_kaggle_unique.loc[idx, 'img_src'] = imgs
        

44679


/opt/anaconda3/envs/udonconda/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-81-8cdc3c3b527e>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_kaggle_unique.loc[idx, 'New_Wiki_Link'] = new_links
<ipython-input-81-8cdc3c3b527e>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_kaggle_unique.loc[idx, 'img_src'] =

44691
44706
44712
44740
44837
44847
44915
44927
44937
44945
44953
44961
44981
45010
45019
45072
45094
45126
45151
45158
45174
45218
45232
45250
45254
45271
45280
45282
45284
45293
45298
45304
45351
45352
45358
45359
45405
45406
45410
45416
45421
45434
45463
45498
45501
45525
45565
45573
45586
45615
45618
45644
45684
45687
45699
45701
45789
45805
45863
45877
45999
46025
46032
46042
46043
46044
46049
46079
46082
46084
46086
46110
46118
46145
46146
46163
46167
46184
46211
46233
46268
46303
46304
46368
46384
46394
46426
46438
46448
46450
46469
46517
46550
46552
46558
46571
46588
46589
46590
46595
46607
46610
46612
46706
46743
46763
46783
46808
46840
46845
46857
46862
46868
46894
46911
46935
46953
46956
46962
46987
46998
47028
47042
47044
47075
47082
47117
47124
47127
47133
47139
47145
47155
47163
47171
47211
47213
47218
47236
47238
47280
47322
47383
47403
47405
47414
47450
47456
47484
47511
47570
47594
47618
47635
47645
47675
47702
47801
47839
47855
47870
47896
47913
47917
47922
47942
4800

In [83]:
check_kaggle_unique[3000:3500].to_csv('check_kaggle_unique_3000.csv')

In [80]:
len(check_kaggle_unique.loc[103123, 'New_Wiki_Link'])

TypeError: object of type 'float' has no len()

In [79]:
check_kaggle_unique

,Art_ID,Title,Artist,Wiki_Link,Style,Genre,Media,tmp,New_Wiki_Link,img_src
Column,,,,,,,,,,
7,Kaggle_49823,Small Bird on a Branch of Kaidozakura,Hiroshige,[https://www.wikiart.org/en/hiroshige/small-bi...,NaN,NaN,NaN,check,[https://www.wikiart.org/en/hiroshige/small-bi...,[https://uploads5.wikiart.org/images/hiroshige...
18,Kaggle_82352,The Garden of Earthly Delights,Hieronymus Bosch,[https://www.wikiart.org/en/hieronymus-bosch/t...,NaN,NaN,NaN,check,[https://www.wikiart.org/en/hieronymus-bosch/t...,[https://uploads2.wikiart.org/images/hieronymu...
20,Kaggle_85864,Untitled,Mark Rothko,[https://www.wikiart.org/en/mark-rothko/untitl...,NaN,NaN,NaN,check,[https://www.wikiart.org/en/mark-rothko/untitl...,[https://uploads3.wikiart.org/images/mark-roth...
37,Kaggle_15715,Self-portrait,Titian,[https://www.wikiart.org/en/titian/self-portra...,NaN,NaN,NaN,check,[https://www.wikiart.org/en/titian/self-portra...,[https://uploads5.wikiart.org/images/titian/se...
38,Kaggle_59486,Self Portrait,Gerrit Dou,[https://www.wikiart.org/en/gerrit-dou/self-po...,NaN,NaN,NaN,check,[https://www.wikiart.org/en/gerrit-dou/self-po...,[https://uploads4.wikiart.org/images/gerrit-do...
...,...,...,...,...,...,...,...,...,...,...
103123,Kaggle_22944,Untitled,Horacio Garcia-Rossi,[https://www.wikiart.org/en/horacio-garcia-ros...,NaN,NaN,NaN,check,NaN,NaN
103137,Kaggle_47270,Alpa,Dieter Roth,[https://www.wikiart.org/en/dieter-roth/alpa-1...,NaN,NaN,NaN,check,NaN,NaN
103179,Kaggle_21218,THE PUTT,Louis Wain,[https://www.wikiart.org/en/louis-wain/the-put...,NaN,NaN,NaN,check,NaN,NaN
